# Imports

In [40]:
import pandas as pd
from hbt_tools import plots as hp
from datetime import datetime as dt
from pathlib import Path
from plotly import graph_objects as go
from plotly.subplots import make_subplots

# Settings

In [41]:
save_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\13 Nachrüstung RÜ\Datenauswertung\Plots")

# Read data

## PLS Exports

In [42]:
inputs_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\13 Nachrüstung RÜ\Datenauswertung\Monatsdaten")
files_list = list(inputs_dir.glob('*.csv'))

date_format = '%d.%m.%Y %H:%M:%S'
data = []
for file in files_list:
    df = pd.read_csv(file, sep=';', header='infer', index_col=0, parse_dates=[0],
        date_format=date_format)
    cols_dict = {}
    for col_name_old in df.columns:
        col_name1 = col_name_old.split('\t')[0]
        col_name2 = col_name_old.split('\t')[1].split(';')[1]
        col_name_new = f'{col_name1} - {col_name2}'
        cols_dict.update({col_name_old: col_name_new})
    df.rename(columns=cols_dict, inplace=True)

    data.append(df)
data = pd.concat(data)

In [43]:
niveaus = data.iloc[:,[0,3,6,9]]

## XLS Exports

In [44]:
inputs_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\13 Nachrüstung RÜ\Datenauswertung\Exports_XLS")
files_list = list(inputs_dir.glob('*.xls'))

data = []
for file in files_list:
    df = pd.read_excel(file, index_col=0, sheet_name='data')
    data.append(df)

# Plots

## PLS

In [45]:
# Niveau aus PLS
fig = go.Figure()
for name in niveaus:
    fig.add_trace(go.Scatter(x=niveaus.index, y=niveaus[name], name=name))

fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Niveau [cm]')
fig.update_layout(title='Daten PLS')

save_path = save_dir / 'Niveaus_PLS_2304-2311.html'
fig.write_html(save_path)

## XLS

In [46]:
# Löchliweg aus XLS
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=data[0].index, y=data[0].iloc[:,4]/10, name='Niveau Löchliweg'), secondary_y=False)
fig.add_trace(go.Scatter(x=data[1].index, y=data[1].iloc[:,3], name='Überlauf Löchliweg'), secondary_y=True)

fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Niveau [cm]', secondary_y=False)
fig.update_yaxes(title='Überlauf [0/1]', secondary_y=True)
fig.update_layout(title='Löchliweg')

save_path = save_dir / 'Löchliweg_XLS_2303-2309.html'
fig.write_html(save_path)

In [47]:
# Wiesengrundstrasse aus XLS
fig = go.Figure()
fig.add_trace(go.Scatter(x=data[2].index, y=data[2].iloc[:,3], name='Überlauf Wiesengrundstrasse'))

fig.update_xaxes(title='Zeit')
fig.update_yaxes(title='Überlauf [0/1]')
fig.update_layout(title='Wiesengrundstrasse')

save_path = save_dir / 'Wiesengrundstrasse_XLS_2303-2310.html'
fig.write_html(save_path)